Camflow Dataset Preprocess for flash

This notebook goes through a list of files representing a dataset generated by camflow and prepared by the Unicorn parser to be reformated and split into batches to be processed by flash

In [6]:
from tqdm import tqdm, trange
import os.path as osp
import time
from math import ceil

In [10]:
Parse_data = True

ds_path = '/home/aabouelk/ds/camflow/'
# benign_date = '10-05-2024'
attack_date = '13-03-2024'
benign_date = '30-04-2024'
chunk_size = 200000
split_size = 100

node_types = {}
edge_types = {}
def show(str):
	print (str + ' ' + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time())))


In [11]:
attack_start_idx = 0
def parse_benign_data():
    show('Start processing.')
    file_idx = 0
    i = 0 
    
    for idx in trange(split_size):
        file_path = osp.join(ds_path, benign_date,f'preprocessed/preprocessed-{idx}.txt')
        f = open(file_path, 'r')

        for line in tqdm(f):

            # Split the large graph into smaller graph each with 1M edges
            if i >= file_idx * chunk_size:
                if file_idx > 0: fw.close()
                fw = open(f'5gcamflow/{benign_date}/'+str(file_idx)+'.txt', 'w')
                file_idx += 1

            src_id, dest_id, types = line.strip('\n').split('\t')
            src_type, dest_type, edge_type, ts = types.split(":")
            temp = []
            temp.append(src_id) # Src ID
            temp.append(src_type) # Src Type
            temp.append(dest_id) # Dest ID 
            temp.append(dest_type) # Dest Type
            temp.append(edge_type) # Edge type
            temp.append(ts) # Logical Timestamp
            fw.write(temp[0]+'\t'+temp[1]+'\t'+temp[2]+'\t'+temp[3]+'\t'+temp[4]+'\t'+temp[5]+'\n')
            node_types[src_type] = node_types.get(src_type, 0) + 1
            node_types[dest_type] = node_types.get(dest_type, 0) + 1
            i += 1
                
        f.close()
    fw.close()
    print(file_idx)
    attack_start_idx = ceil(file_idx / 100) * 100
    print(attack_start_idx)


In [12]:
parse_benign_data()


Start processing. 2024-07-06 08:59:02


873850it [00:01, 482152.42it/s], ?it/s]
887238it [00:01, 505163.62it/s]2:59,  1.82s/it]
894784it [00:01, 684649.41it/s]2:54,  1.79s/it]
893226it [00:01, 686604.84it/s]2:32,  1.57s/it]
893687it [00:01, 691869.43it/s]2:20,  1.46s/it]
893366it [00:01, 688803.04it/s]2:13,  1.40s/it]
891844it [00:01, 683272.26it/s]2:08,  1.37s/it]
892549it [00:01, 689932.97it/s]2:05,  1.35s/it]
892781it [00:01, 691296.39it/s]2:02,  1.33s/it]
891789it [00:01, 680734.18it/s]2:00,  1.32s/it]
889954it [00:01, 686545.38it/s]01:58,  1.32s/it]
892337it [00:01, 584536.56it/s]01:56,  1.31s/it]
893238it [00:01, 620801.01it/s]02:01,  1.38s/it]
891177it [00:01, 677188.02it/s]02:01,  1.40s/it]
892169it [00:01, 651643.23it/s]01:58,  1.38s/it]
892347it [00:01, 671859.71it/s]01:56,  1.37s/it]
891697it [00:01, 674356.93it/s]01:54,  1.36s/it]
892703it [00:01, 680194.58it/s]01:52,  1.35s/it]
893776it [00:01, 687332.45it/s]01:49,  1.34s/it]
891308it [00:01, 691718.73it/s]01:47,  1.33s/it]
892260it [00:01, 687638.70it/s]01:45, 

446
500


In [13]:
attack_start_idx = 500

In [14]:
    
def parse_attack_data():
    file_idx = attack_start_idx+1
    print(file_idx)
    i = 0 
    file_path = osp.join(ds_path, attack_date, f'privesc/preprocessed.txt')
    f = open(file_path, 'r')

    fw = open(f'5gcamflow/{benign_date}/{file_idx}.txt', 'w')
    for line in tqdm(f):
        # Split the large graph into smaller graph each with 1M edges
        if i >= (file_idx-attack_start_idx) * chunk_size:
            if file_idx > 0: fw.close()
            fw = open(f'5gcamflow/{benign_date}/{file_idx}.txt', 'w')
            file_idx += 1

        src_id, dest_id, types = line.strip('\n').split('\t')
        src_type, dest_type, edge_type, ts = types.split(":")
        temp = []
        temp.append(src_id) # Src ID
        temp.append(src_type) # Src Type
        temp.append(dest_id) # Dest ID 
        temp.append(dest_type) # Dest Type
        temp.append(edge_type) # Edge type
        temp.append(ts) # Logical Timestamp
        fw.write(temp[0]+'\t'+temp[1]+'\t'+temp[2]+'\t'+temp[3]+'\t'+temp[4]+'\t'+temp[5]+'\n')
        node_types[src_type] = node_types.get(src_type, 0) + 1
        node_types[dest_type] = node_types.get(dest_type, 0) + 1
        i += 1
            
    f.close()
    fw.close()
    show('Done.')


In [15]:
parse_attack_data()

501


951538it [00:01, 481256.29it/s]

Done. 2024-07-06 09:01:29


In [24]:
from pprint import pprint

# sorted_edge_types = {k:v for k, v in sorted(edge_types.items(), key=lambda item: item[1])}
# pprint(sorted_edge_types)
pprint(node_types)

{'address': 59766,
 'argv': 223812,
 'block': 54555,
 'file': 30822393,
 'iattr': 73734,
 'link': 1783680,
 'path': 656420,
 'pipe': 15761050,
 'process_memory': 71387160,
 'socket': 27633980,
 'task': 209812044,
 'xattr': 34346}
